## pipeline

In [1]:
import dgl
import networkx as nx
import numpy as np
import torch

import torch.nn as nn

from statistics import mean

from func.run_pipeline_super_vox import get_outlayer_of_a_3d_shape, get_crop_by_pixel_val
from func.ultis import load_obj

from func.graph_learning import SuperVoxToNxGraph, VoxelGraphDataset


Using backend: pytorch


In [2]:
# load graphs
from func.ultis import load_obj

# graphs = load_obj("graphs_dataset_train")
graphs = load_obj("ovules_training_graphs/graphs_dataset_train_ovules_retrained_total")
# test_graphs = load_obj("ovules_training_graphs/graphs_dataset_train_ovules_retrained_skript_testset")
model_save_path = "output/graph_model_focal_ovules_retrained.pt"

In [3]:
import random
random.seed(0)
dgl.seed(0)
torch.manual_seed(0)
random.shuffle(graphs)

In [4]:
dataset = VoxelGraphDataset(graphs, with_edge_weights=True)

g = dataset[0]


/opt/anaconda3/envs/CellSeg3D_torch/lib/python3.8/site-packages/dgl/backend/pytorch/tensor.py:40: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  return th.as_tensor(data, dtype=dtype)


# TODO probably should normalize features!!!!

In [5]:
num_examples = len(dataset)
num_train = int(num_examples)

In [ ]:
from func.graph_models import GCN

model = GCN(3, num_classes=1)
CELoss = nn.CrossEntropyLoss()

# training
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
best_val_acc = 0

from torchmetrics import F1Score
from torchvision.ops import sigmoid_focal_loss

f1 = F1Score(num_classes=1, average='weighted')

epoch_loss = []
epoch_val_loss = []
epoch_accuracy = []

epoch_f1score = []
epoch_f1score_val = []

epoch_accuracy_val = []
# best_val_acc = 0
best_val_loss = 1000

for e in range(300):
    alpha = 0.23
    for graph_number in range(len(dataset)):
        # Forward
        model.train()
        sample_graph = dataset[graph_number]
        features = sample_graph.ndata['feat']
        labels = sample_graph.ndata['label']
        train_mask = sample_graph.ndata['train_mask']
        val_mask = sample_graph.ndata['val_mask']

        # create class weights
        number_positives = torch.count_nonzero(labels)
        percentage_positives = number_positives / len(labels)
        percentage_negatives = 1 - percentage_positives

        # CELoss = nn.CrossEntropyLoss(weight=weights)
        train_mask = sample_graph.ndata['train_mask']
        val_mask = sample_graph.ndata['val_mask']
        logits = model(sample_graph, features)
        model_output = torch.sigmoid(logits)

        # Compute prediction
        pred = (model_output > 0.5).type(torch.FloatTensor)
        loss = sigmoid_focal_loss(torch.squeeze(logits[train_mask].type(torch.FloatTensor)), labels[train_mask].type(torch.FloatTensor), alpha=alpha, reduction="mean")



        epoch_loss.append(loss.item())

        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()

        train_f1_score = f1(pred[train_mask], labels[train_mask])
        val_f1_score = f1(pred[val_mask], labels[val_mask])


        epoch_accuracy.append(train_acc.item())
        epoch_accuracy_val.append(val_acc.item())

        epoch_f1score.append(train_f1_score.item())
        epoch_f1score_val.append(val_f1_score.item())


        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            logits = model(sample_graph, features)
            # val_loss = CELoss(logits[val_mask], labels[val_mask])
            val_loss = sigmoid_focal_loss(torch.squeeze(logits[val_mask].type(torch.FloatTensor)), labels[val_mask].type(torch.FloatTensor), alpha=alpha, reduction="mean")
            epoch_val_loss.append(val_loss.item())
        model.train()

    if e % 5 == 0:
        print('In epoch {}, loss: {:.5f}, val loss: {:.5f}, accuracy: {:.3f}, val accuracy: {:.3f}, f1score: {:.3f}, val f1score: {:.3f}'.format(
            e, mean(epoch_loss), mean(epoch_val_loss), mean(epoch_accuracy), mean(epoch_accuracy_val), mean(epoch_f1score), mean(epoch_f1score_val)))

        #if mean(epoch_accuracy_val) >= best_val_acc:
        if mean(epoch_val_loss) <= best_val_loss:
            print("new best val loss")
            torch.save(model.state_dict(), model_save_path)
            best_val_loss = mean(epoch_val_loss)
        epoch_loss = []
        epoch_val_loss = []
        epoch_accuracy = []

        epoch_accuracy_val = []
        epoch_f1score_val = []


/opt/anaconda3/envs/CellSeg3D_torch/lib/python3.8/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


# Train on GPU
g = g.to('cuda')
model = GCN(1, 16, dataset.num_classes).to('cuda')
train(g, model)

In [ ]:
g

In [ ]:

g.ndata['feat'].shape

In [ ]:
model.eval()
with torch.no_grad():
    predictions = (model(g, g.ndata['feat']) > 0.5)
    # pred = (model_output > 0.5).type(torch.FloatTensor)

In [ ]:
np.unique(predictions)

In [ ]:
labels = g.ndata['label']
np.unique(labels)

In [ ]:
len(labels[labels==1])

In [ ]:
len(labels[labels==0])

In [ ]:
len(predictions[predictions==1])

In [ ]:
len(predictions[predictions==0])

In [ ]:
graph_model = GCN(3, num_classes=1)
checkpoint_graph = 'output/graph_model_focal.pt'
graph_model.load_state_dict(torch.load(checkpoint_graph))
graph_model.eval()

In [ ]:
predictions = (graph_model(g, g.ndata['feat']) > 0.5).numpy()


In [ ]:

len(predictions[predictions==1])

In [ ]:
len(predictions[predictions==0])